In [1]:
from urllib.request import urlopen  # Library for urlopen
from bs4 import BeautifulSoup  # Library for html parser (scraper), lxml is also nice
import pandas as pd
import numpy as np
import re
import sys
sys.path.append('..') 
from uni_cache.cache_function import cache_function
import pymysql
import collections
import mysql_credits

In [2]:
# This folder should be edited according to this project path on yours computer
project_folder = '/home/bogdan/PycharmProjects/universities_sentiment/'
cache_folder = project_folder + 'cache/'
#site = 'https://www.studentsreview.com/CA/CIT_comments.html'


connection = pymysql.connect(
    host=mysql_credits.db_host,
    user=mysql_credits.db_user,
    password=mysql_credits.db_password,
    db=mysql_credits.db,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)


In [1]:
import time

In [2]:
def parsing_function(site_tree):
    # Преобразовываем файл к объектному типу библиотеки BeautifulSoup
    site_bs = BeautifulSoup(site_tree, "html.parser")
    # Ищем все вхождения ссылок с главной страницы на статьи
    site_titles = site_bs.find_all('div', attrs={"class":"reviewcomment"})
    
    
    articles_data = []
    for site_title in site_titles:
        #if site_title.find('div', attrs={"class":"submitted"}):
            # Создаём что-то вроде ассоциативного массива
        article_meta_data = collections.OrderedDict()
            # Херачим в него ссылки статьи
        article_meta_data['article_url'] = site_title.find('a', attrs={"class":"readmore"}).get('href')
            # Херачим в него заголовки статьи readmore intellect negativetext
        if site_title.find('span', attrs={"class": "reviewtitle"}):
            article_meta_data['article_title'] = re.sub(' +',' ',site_title.find('span', attrs={"class": "reviewtitle"}).get_text().replace("\\n", "")) 
        else:
            article_meta_data['article_title']='no_article_title'
            # Херачим в него дату отзыва
        if site_title.find('em'):
            article_meta_data['article_date'] =re.sub(' +',' ', site_title.find('em').get_text().replace("\\n", "")) 
        else:
            article_meta_data['article_date']='2017-10-10'
            #print(site_title.get_text())

                

            # Вкорячиваем этот "ассоциативный массив" в просто массив
        articles_data.append(article_meta_data)
    return articles_data

In [4]:
with open('USA.txt', 'r', encoding='windows-1251') as f:
    content = f.read().splitlines()
print(content[0])

http://www.studentsreview.com/CA/CIT_comments.html


In [5]:
dict_uni={}
dict_stars={}
for i, item in enumerate([i.lower() for i in ['Negative','Neutral','Positive','Advice']]):
    print(i,item)
    dict_stars[item]=i+1

0 negative
1 neutral
2 positive
3 advice


In [12]:
list(dict_stars.keys())

['negative', 'neutral', 'positive', 'advice']

In [15]:
np.in1d(list(dict_stars.keys()), page_article_bs.find("div", attrs={"id": "wide"}).attrs['class'][-1]).sum()!=0

True

In [8]:
page_article_bs.find("div", attrs={"id": "wide"}).attrs['class'][-1]

'negative'

In [21]:
site_news_section

'?page=18'

In [20]:
uni

'http://www.studentsreview.com/CT/YU_comments.html'

In [26]:
#n=1
for uni in content[8:]:
    print(uni)
    # Преобразовываем файл к объектному типу библиотеки BeautifulSoup
    site = uni
    main_site=cache_function(site)
    max_page = BeautifulSoup(main_site, "html.parser").find_all('div', attrs={"class":"pagenav"})[-1].find_all('a')[-1].get_text()
    name_uni = BeautifulSoup(main_site, "html.parser").find('h1', attrs={"id":"uname"}).get_text()
    #dict_uni[name_uni]=n
    max_page=int(max_page)
    print(max_page)
    for i in range(1,max_page+1):
        site_news_section = '?page='+str(i)
        site_tree = cache_function(site + site_news_section)
        articles = parsing_function(site_tree)
        #dict_uni[name_uni]=n
        print(len(articles))
        for article in articles:
            site_tree = cache_function(article['article_url'])
            #print()
            page_article_bs = BeautifulSoup(site_tree, "html.parser")
            #page_article = page_article_bs.find_all('div', class_='field-item even')
            if page_article_bs.find("div", attrs={"id": "wide"}):
                article_content = page_article_bs.find("div", attrs={"id": "wide"}).get_text()
                #print(article_content)
            else:
                article_content = 'no_review_text'
             # Херачим в него рейтинг отзыва
            if np.in1d(list(dict_stars.keys()), page_article_bs.find("div", attrs={"id": "wide"}).attrs['class'][-1]).sum()!=0:
                review_stars = dict_stars[page_article_bs.find("div", attrs={"id": "wide"}).attrs['class'][-1]]#page_article_bs.find_all('div', attrs={"class":"snapshot"})[-1].find('span').get_text()
                #print(review_stars)
            else:
                review_stars = dict_stars['neutral']
            emoji_pattern = re.compile("["
                                    u"\U0001F600-\U0001F64F"  # emoticons
                                    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                       "]+", flags=re.UNICODE)
            article_content = emoji_pattern.sub(r'', article_content)

            try:
                with connection.cursor() as cursor:
                    # Create a new record
                    #INSERT INTO `article` (`article_title`, `article_text`, `article_url`, `article_categories`) VALUES (%s, %s, %s, 'null');
                    sql = '''
                    INSERT INTO `article` 
                    (`article_pub_date`, `article_title`, `article_text`, 
                    `article_url`, `article_rating`, `article_uni`, `uni_site_id`)
                    VALUES (%s, %s, %s,%s, %s, %s, %s);
                    '''

                    #sql="INSERT INTO `article` (`article_pub_date`, `article_title`, `article_text`, `article_url`, `article_rating`, `article_uni`, `uni_site_id`) VALUES ('"+article['article_date']+"','"+article['article_title']+ "', '"+article_content+"', '"+article['article_url']+"', 'null', 'null',null');"

                    #ins=
                    cursor.execute(sql, (str(pd.to_datetime(article['article_date'])), article['article_title'], article_content, article['article_url'], review_stars,name_uni,0))
                    #cursor.execute(sql)
                # connection is not autocommit by default. So you must commit to save
                # your changes.
                connection.commit()
            finally:
                print('finally')
    
    #n+=1

http://www.studentsreview.com/MD/JHU_comments.html
Using web page from internet...
52
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet..

3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
fi

finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
fi

3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
fi

finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
fi

...

finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet

finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet

finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from cache...
finally
Using web page from cache...
finally
Using web page from cache...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Usi

finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
fi

3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
finally
Using web page from internet...
3
Using web page from internet...
finally
Using web page from internet...
fi